In [43]:
from algorithm1 import algorithm1,random,TabuList,copy,map_price_demand,reset_update_prices,cmp_to_key,reset_students,time
from algorithm2 import algorithm2,csp_mapping,copy,math,cmp_to_key
from algorithm3 import algorithm3,mapping_csp


In [49]:
a = Course(name='a', price=2.4, capacity=0, max_capacity=5)
b = Course(name='b', price=5, capacity=0, max_capacity=4)
c = Course(name='c', price=10.4, capacity=0, max_capacity=5)
courses = [a, b, c]
s1 = Student(name='s1', budget=18, year=1, courses=[], preferences=([c, b]))
s2 = Student(name='s2', budget=18, year=1, courses=[], preferences=([b, c, a]))
s3 = Student(name='s3', budget=18, year=1, courses=[], preferences=([b, a]))
s4 = Student(name='s4', budget=18, year=1, courses=[], preferences=([a, c]))
students = [s1, s2, s3, s4]


print(str(s1))

In [45]:
i = 1

courses = []
for k in range(10*i):
    name = chr(ord('a') + i)
    price = random.randint(3, 15)
    capacity = 0
    max_capacity = random.randint(3, 10)
    courses.append(Course(name=name, price=price,
                          capacity=capacity, max_capacity=max_capacity))

students = []
for p in range(40*i):
    name = 's' + str(i+1)
    budget = random.randint(15, 25)
    year = random.randint(1, 4)
    preferences = random.sample(courses, random.randint(3, 7))
    students.append(Student(name=name, budget=budget,
                            year=year, courses=courses, preferences=preferences))


def process_copy(student: Student, i: int):
    copy_student = copy.deepcopy(student)
    copy_student.preferences = student.preferences[:]
    return copy_student, i


In [47]:

wow = []
import concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(process_copy, students[i], i) for i in range(len(students))]
    # return each result as soon as it is completed:
    for future in concurrent.futures.as_completed(futures):
        wow.append(future.result())
        # wow[future.result()[0]] = future.result()[1]
# for s in students:
#     wow.append(process_copy(s))
# wow = [process_copy(s) for s in students]
wow.sort(key=lambda x: x[1])
for s in students:
    print(str(s))
# print([s for s in students])
wow = [w[0] for w in wow] 
for s in wow:
    print(s.__str__)
# print([s for s in wow])

# print(students)

# max_budget = 18
# price_vector1 = algorithm1(courses=courses,max_budget=max_budget, seed=3, students=students,time_to=1)
# map_price_demand(price_vector=price_vector1,courses=courses,max_budget=max_budget,students=students)
# price_vector=[course.price for course in courses]
# p_scalar = (max_budget- 0.1)
# eps=0.5


<method-wrapper '__str__' of Student object at 0x7f04a8b699c0>
<method-wrapper '__str__' of Student object at 0x7f048af2e110>
<method-wrapper '__str__' of Student object at 0x7f04a80fc850>
<method-wrapper '__str__' of Student object at 0x7f048af6b8e0>
<method-wrapper '__str__' of Student object at 0x7f048af6a6b0>
<method-wrapper '__str__' of Student object at 0x7f04a81f46a0>
<method-wrapper '__str__' of Student object at 0x7f04a81f5c90>
<method-wrapper '__str__' of Student object at 0x7f04a80fd240>
<method-wrapper '__str__' of Student object at 0x7f04a80fd1b0>
<method-wrapper '__str__' of Student object at 0x7f048af06f80>
<method-wrapper '__str__' of Student object at 0x7f04a811c3d0>
<method-wrapper '__str__' of Student object at 0x7f04a811c6a0>
<method-wrapper '__str__' of Student object at 0x7f04a811d5a0>
<method-wrapper '__str__' of Student object at 0x7f04a8b86fe0>
<method-wrapper '__str__' of Student object at 0x7f048af201c0>
<method-wrapper '__str__' of Student object at 0x7f04a8

In [98]:
print(students[2].budget)

3.6000000000000014


In [103]:
import json

def save_to_json(file_name: str) -> bool:
    """
    Saves the graph in JSON format to a file
    @param file_name: The path to the out file
    @return: True if the save was successful, False o.w.
    """
    with open(file_name, 'w') as file:  # Write to file
        try:
            d = {"students": [], "courses": []}
            for student in students:
                d["students"].append({
                    "name": student.name,
                    "budget": student.budget,
                    "preferences": json.dumps([course.name for course in student.preferences]),
                    "courses": json.dumps([course.name for course in student.courses]),
                    "year": student.year
                })
            for course in courses:
                d["courses"].append({
                    "name": course.name,
                    "price": course.price,
                    "max_capacity": course.max_capacity,
                    "capacity": course.capacity
                })
            d["price_vector"] = json.dumps([course.price for course in courses])
            d["maximum"] = p_scalar
            d["eps"] = eps
            json.dump(d, file)  # Stores d into file
            return True
        except Exception as e:
            print(e)
            return False
        finally:
            file.close()
    # raise NotImplementedError


In [104]:
save_to_json(file_name="wow.txt")

True

In [110]:
def load_from_json(file_name: str) -> bool:
    """
    Loads a graph from a json file.
    @param file_name: The path to the json file
    @returns True if the loading was successful, False o.w.
    """
    try:
        with open(file_name) as file:
            s = json.load(file)
            courses = []
            for course in s["courses"]:
                courses.append(
                    Course(name=course["name"], price = course["price"],
                        max_capacity= course["max_capacity"], capacity=course["capacity"])
                )
            helper = {course.name:course for course in courses}

            students = []
            for student in s["students"]:
                # pref = student["preferences"].split(",")
                preferences=json.loads(student["preferences"])
                students.append(
                    Student(name=student["name"], budget=student["budget"],
                        preferences=[helper[pre] for pre in preferences],
                        courses=[]
                    )
                )
            price_vector = json.loads(s["price_vector"])
            maximum = s["maximum"]
            eps = s["eps"]
            return True
    except Exception as e:
        print(e)
        return False
    finally:
        file.close()


In [111]:
load_from_json("wow.txt")


'price_vector'


False

In [84]:
import random
import time
from Course import Course
from Student import Student
from algorithm1 import algorithm1, Course, Student, random, TabuList, copy, map_price_demand, reset_update_prices, cmp_to_key, reset_students, time


In [85]:
random.seed(3)
times = []
for i in range(1, 2):
    start_time = time.time()
    # Generate 10 courses
    courses = []
    for i in range(10*i):
        name = chr(ord('a') + i)
        price = random.randint(3, 15)
        capacity = 0
        max_capacity = random.randint(3, 10)
        courses.append(Course(name=name, price=price,
                       capacity=capacity, max_capacity=max_capacity))
                       capacity = capacity, max_capacity = max_capacity))

        # Generate 40 students
    students=[]
    for i in range(40*i):
        name='s' + str(i+1)
        budget=random.randint(15, 25)
        year=random.randint(1, 4)
        students.append(Student(name = name, budget = budget,
                        year = year, courses = courses, preferences = preferences))
        students.append(Student(name = name, budget = budget,
                        year = year, courses = courses, preferences = preferences))
    times.append((time.time() - start_time))


IndentationError: unexpected indent (326770922.py, line 14)

In [ ]:
# start_time = time.time()
max_budget = 25
price_vector1 = algorithm1(
    courses=courses, max_budget=max_budget, seed=3, students=students, time_to=10)
# print(time.time() - start_time)
# print(time.time() - start_time)


best error is 79264 
best error is 62215 
best error is 43579 
best error is 39274 


In [ ]:
# start_time = time.time()
max_budget = 25
price_vector1 = algorithm1(
    courses=courses, max_budget=max_budget, seed=3, students=students, time_to=10)
# print(time.time() - start_time)
# print(time.time() - start_time)


best error is 76532 
best error is 57433 
best error is 39816 
best error is 37061 
